In [ ]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

import warnings   # To avoid warning messages in the code run
warnings.filterwarnings("ignore")

In [ ]:
# The data, split between train and test sets:
(x_train_, y_train_), (x_test_, y_test_) = cifar10.load_data()
# delete C:/Users/Your_UserName/.keras/datasets/
print('x_train_ shape:', x_train_.shape)
print('x_test_ shape:', x_test_.shape)
print('y_train_ shape:', y_train_.shape)
print('y_test_ shape:', y_test_.shape)
print('y_train_:', y_train_)
print('y_test_:', y_test_)
# airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, & trucks

In [ ]:
x_train, __x, y_train, __y = train_test_split(x_train_, y_train_, train_size=7500, stratify=y_train_, random_state=0)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('y_train:', y_train)

In [ ]:
x_val, __x, y_val, __y = train_test_split(__x, __y, train_size=1250, stratify=__y, random_state=0)
print('x_val shape:', x_val.shape)
print('y_val shape:', y_val.shape)

In [ ]:
x_test, __x, y_test, __y = train_test_split(x_test_, y_test_, train_size=1250, stratify=y_test_, random_state=0)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

In [ ]:
plt.imshow(x_train[5])
plt.show()
print('Label:', y_train[5][0])

In [ ]:
num_classes = 10

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train shape:', y_train.shape)
print('y_val shape:', y_val.shape)
print('y_test shape:', y_test.shape)

print('y_train:', y_train)
print('y_val:', y_val)
print('y_test:', y_test)

In [ ]:
model = Sequential()
model.add(Conv2D(8, (3, 3), input_shape=(32, 32, 3))) # 30 30 8
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # 15 15 4
# model.add(Dropout(0.25))

model.add(Conv2D(16, (4, 4))) # 12 12 8
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # 6 6 8
# model.add(Dropout(0.5))

model.add(Conv2D(32, (5, 5))) # 2 2 16
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # 1 1 16
# model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(32))
model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# initiate Adam optimizer
opt = keras.optimizers.adam(lr=0.005)

# Let's train the model using Adam
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_val /= 255.0
x_test /= 255.0

if not os.path.exists('saved_cnn_models'):
    os.mkdir('saved_cnn_models')

num_epochs = 16
filepath = 'saved_cnn_models/model-{epoch:05d}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(x=x_train, y=y_train, batch_size=250, epochs=num_epochs, verbose=1, callbacks=callbacks_list,
              validation_data=(x_val, y_val))

In [ ]:
history.history

In [ ]:
plt.plot(range(num_epochs-1), history.history['loss'][1:], label='train_loss')
plt.plot(range(num_epochs-1), history.history['val_loss'][:-1], label='val_loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(range(num_epochs-1), history.history['acc'][1:], label='train_acc')
plt.plot(range(num_epochs-1), history.history['val_acc'][:-1], label='val_acc')
plt.legend()
plt.grid()
plt.show()

In [ ]:
print('Minimum Validation Loss:', np.min(history.history['val_loss']))
print('Best epoch corresponding to Minimum Validation Loss:', np.argmin(history.history['val_loss'])+1)

In [ ]:
model_best = load_model('saved_cnn_models/model-00010.h5')

In [ ]:
scores = model_best.evaluate(x_test, y_test, verbose=1)

In [ ]:
scores

In [ ]:
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])